In [1]:
from __future__ import annotations

import base64
from email.message import EmailMessage
from pathlib import Path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# If you want "read + send" with one scope, gmail.modify works.
# (messages.send accepts gmail.modify/gmail.compose/gmail.send/etc.)  :contentReference[oaicite:5]{index=5}
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

CREDS_JSON = Path("credentials.json")
TOKEN_JSON = Path("token.json")

def get_service():
    creds = None
    if TOKEN_JSON.exists():
        creds = Credentials.from_authorized_user_file(TOKEN_JSON, SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDS_JSON, SCOPES)
            creds = flow.run_local_server(port=0)
        TOKEN_JSON.write_text(creds.to_json())

    return build("gmail", "v1", credentials=creds)

def list_unread(limit: int = 10):
    svc = get_service()
    # q supports Gmail search operators (e.g. "is:unread from:boss newer_than:2d") :contentReference[oaicite:6]{index=6}
    res = svc.users().messages().list(userId="me", q="is:unread", maxResults=limit).execute()
    return [m["id"] for m in res.get("messages", [])]

def get_snippet(msg_id: str) -> str:
    svc = get_service()
    msg = svc.users().messages().get(userId="me", id=msg_id, format="metadata").execute()
    return msg.get("snippet", "")

def send_email(to: str, subject: str, body: str):
    svc = get_service()

    m = EmailMessage()
    m["To"] = to
    m["Subject"] = subject
    m.set_content(body)

    raw = base64.urlsafe_b64encode(m.as_bytes()).decode("utf-8")
    # Gmail API sends base64url-encoded RFC 2822 MIME in "raw" :contentReference[oaicite:7]{index=7}
    svc.users().messages().send(userId="me", body={"raw": raw}).execute()

if __name__ == "__main__":
    ids = list_unread(5)
    for mid in ids:
        print(mid, get_snippet(mid))

    # send_email("someone@example.com", "Hello from a script", "This was sent via Gmail API.")


FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'